In [35]:
import regex

from pathlib import Path

In [37]:
def find_occurences(pattern: str, limit: int = None):
    out = {}
    for ustawa in list(Path("./ustawy/").iterdir())[:limit]:
        found = regex.findall(pattern=pattern, string=ustawa.read_text(encoding="utf8"))
        out[ustawa.name] = found
    return out

In [73]:
# Przydatny regex, dopasowujący wystąpienie jednostki redakcyjnej 
# artykuł, paragraf, ustęp, punkt, litera
# wraz z zakresem (np. art. 1-3)
# grupy są non-capturing, żeby regex nie zwracał krotek
jedn_regex = r"(?:art\. \d+\w*(?:-\d+\w*)*|§ \d+(?:-\d+)*|ust\. \d+\w*(?:-\d+\w*)*|pkt \d+\w*(?:-\d+\w*)*|lit\.\w+\)(?:-\w+\))*)"
find_occurences(jedn_regex, 1)["1993_599.txt"][32:42]

['ust. 2a',
 'ust. 2',
 'ust. 3',
 'ust. 1',
 'art. 14',
 'art. 7',
 'ust. 1',
 'art. 47',
 'art. 21',
 'pkt 2-4']

### 1. Dla każdej ustawy policz wszystkie wystąpienie następujących nowelizacji:
#### a. dodanie jednostki (e.g. dodaje się ust. 5a)
Spostrzeżenia:
- na początku dodania występuje równoważnik zdania zawierający synonim "dodaje się" (składa się z liter, liczb, kropek, spacji, § - bo mogą być wypisane jednostki redakcyjne),
- następnie określona jest dodawana jednostka redakcyjna (lub jednostki),
- na końcu występuje "w brzmieniu:".

In [88]:
r1a = fr"[\w\d\.§ ]+{jedn_regex}(?: i {jedn_regex}| i \d)? w brzmieniu:"
find_occurences(pattern=r1a, limit=3)

{'1993_599.txt': [' dodaje się ust. 2 w brzmieniu:',
  ' po ust. 8 dodaje się ust. 8a w brzmieniu:',
  ' po ust. 5 dodaje się ust. 5a w brzmieniu:',
  ' po ust. 2 dodaje się ust. 2a w brzmieniu:',
  ' w art. 20 po ust. 3 dodaje się ust. 4 i 5 w brzmieniu:',
  ' po pkt 3 dodaje się pkt 4 w brzmieniu:',
  ' po ust. 3 dodaje się ust. 4 w brzmieniu:',
  ' dodaje się ust. 5 w brzmieniu:',
  ' po pkt 3 dodaje się pkt 4 i 5 w brzmieniu:',
  ' dodaje się ust. 3 w brzmieniu:'],
 '1993_602.txt': [],
 '1993_645.txt': []}

#### b. usunięcie jednostki (np. w art. 10 ust. 1 pkt 8 skreśla się)

#### c. zmianę jednostki (np. art. 5 otrzymuje brzmienie)